In [45]:
import pandas as pd
import numpy as np
import os
import subprocess
import matplotlib.pyplot as plt
import statsmodels
%matplotlib inline
plt.style.use("dark_background")

In [6]:
print os.getcwd()
print os.listdir(os.getcwd())


/home/chris/Documents/bigforecast/FeatureEngineering/GDELT
['GDELT2-exploration.ipynb', 'get_GDELT.py', '.ipynb_checkpoints', '20170718201500.export.CSV', '.#get_GDELT.py', '20170718201500.export.CSV.zip', 'GDELT-Event_Codebook-V2.0.pdf']


In [94]:
with open("feature_names.txt", "r") as f:
    cols = f.readlines()

cols = list(map(lambda s: s.replace("\n", ""), cols))

In [95]:
gdelt = pd.read_csv("20170718201500.export.CSV", sep = "\t", header = None)
gdelt.columns = cols
#df.head()
gdelt.head()

   GlobalEventID       Day  MonthYear  Year  FractionDate Actor1Code  \
0      674078723  20160718     201607  2016     2016.5425        NaN   
1      674078724  20160718     201607  2016     2016.5425        AGR   
2      674078725  20160718     201607  2016     2016.5425        AUS   
3      674078726  20160718     201607  2016     2016.5425        AUS   
4      674078727  20160718     201607  2016     2016.5425        BUS   

    Actor1Name Actor1CountryCode Actor1KnownGroupCode Actor1EthnicCode  \
0          NaN               NaN                  NaN              NaN   
1       FARMER               NaN                  NaN              NaN   
2    AUSTRALIA               AUS                  NaN              NaN   
3    AUSTRALIA               AUS                  NaN              NaN   
4  CORPORATION               NaN                  NaN              NaN   

                         ...                         ActionGeo_Type  \
0                        ...                       

In [92]:
gdelt.shape

(3658, 61)

In [12]:
prices = pd.read_csv("~/Documents/bigforecast/oil_price.csv", sep = "\t")
prices.columns = ["Date", "Price"]
prices["Date"] = pd.to_datetime(prices["Date"])
prices.tail()

           Date  Price
8061 2017-07-11  45.06
8062 2017-07-12  45.48
8063 2017-07-13  46.06
8064 2017-07-14  46.53
8065 2017-07-17  46.02

In [66]:
%load_ext autoreload
%autoreload 2


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [69]:
from datetime import date, timedelta
from get_GDELT import get_file

In [56]:
def date_range(start_date, end_date):
    for n in range(int((end_date - start_date).days)):
        yield start_date + timedelta(n)

In [86]:
start = date(2017, 7, 1)
end = date(2017, 7, 4)
from get_GDELT import get_file
for single_date in date_range(start, end):
    get_file(file_date = single_date, version = 1, time = None)

2017-07-03
http://data.gdeltproject.org/events/20170703.export.CSV.zip


2017-07-02
http://data.gdeltproject.org/events/20170702.export.CSV.zip


2017-07-01
http://data.gdeltproject.org/events/20170701.export.CSV.zip


In [77]:
d = date(2017, 7, 18)
print(d.isoformat())
print(type(d.isoformat()))
print(type(d.isoformat()))

2017-07-18
<class 'str'>


In [150]:
df = pd.concat([pd.read_csv("data/" + file_name,
                            sep = "\t",
                            compression = "zip",
                            low_memory = False,
                            names = cols) for file_name in os.listdir(os.getcwd() + "/data/")])
df.shape


(382368, 61)

In [177]:
use_cols = ["Day", "GoldsteinScale"]
avg_days = df[use_cols].groupby("Day").agg(["mean", "median", "var"])
avg_days.columns.values
avg_days.reset_index(inplace = True)

avg_days.columns = ["Date", "mean", "median", "var"]
avg_days["Date"] = pd.to_datetime(avg_days["Date"].astype('str'))

#avg_days.set_index("Day", inplace = True)
avg_days.head()


        Date      mean  median        var
0 2007-07-04 -0.273077    1.90  36.176446
1 2007-07-05  0.742857    1.00  18.702857
2 2007-07-06  0.075000    1.45  25.848667
3 2016-07-01  0.631647    1.90  28.443099
4 2016-07-02  0.703977    1.90  27.183169

In [149]:
avg_days.plot(x = "Day", y = "mean", kind = "bar")

In [129]:
oil_price = pd.read_csv("~/Documents/bigforecast/oil_price.csv",
                        sep = "\t",
                        names = ["Date", "Price"],
                        parse_dates = ["Date"])
oil_price.head()

        Date  Price
0 1986-01-02  25.56
1 1986-01-03  26.00
2 1986-01-06  26.53
3 1986-01-07  25.85
4 1986-01-08  25.87

In [180]:
avg_days.head()

        Date      mean  median        var
0 2007-07-04 -0.273077    1.90  36.176446
1 2007-07-05  0.742857    1.00  18.702857
2 2007-07-06  0.075000    1.45  25.848667
3 2016-07-01  0.631647    1.90  28.443099
4 2016-07-02  0.703977    1.90  27.183169

In [184]:
oil_price.tail(10)

           Date  Price
8057 2017-07-04    NaN
8058 2017-07-05  45.11
8059 2017-07-06  45.52
8060 2017-07-07  44.25
8061 2017-07-10  44.40
8062 2017-07-11  45.06
8063 2017-07-12  45.48
8064 2017-07-13  46.06
8065 2017-07-14  46.53
8066 2017-07-17  46.02

In [182]:
avg_days.join(oil_price, on = "Date", how = "inner", lsuffix = "_left")

Empty DataFrame
Columns: [Date_left, mean, median, var, Date, Price]
Index: []